# ADM-HW4: GROUP #10

#### Xavier Del Giudice


| STUDENT |   ID    |                 E-mail                  |
| :-: |:-------:|:---------------------------------------:|
| Xavier Del Giudice | 1967219 | delgiudice.1967219@studenti.uniroma1.it |
|  Valeria Avino| 1905974 |   avino.1905974@studenti.uniroma1.it                                      |
|  |  |                       |

## Import and Install required packages

In [ ]:
%pip install pandas numpy

In [2]:
import pandas as pd
import numpy as np

# 1. Recommendation System with LSH

## 1.1 Data Preparation

Reading the dataset and checking (if any) missing data.

In [23]:
#reading from our dataset
genome_scores = pd.read_csv('MovieLens/genome_scores.csv')
genome_tags = pd.read_csv('MovieLens/genome_tags.csv')
link = pd.read_csv('MovieLens/link.csv')
movie = pd.read_csv('MovieLens/movie.csv')
rating = pd.read_csv('MovieLens/rating.csv')
tags = pd.read_csv('MovieLens/tag.csv')

In [24]:
genome_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 268.0 MB


In [25]:
#checking missing values
missing_values = genome_scores.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [26]:
genome_scores.duplicated().sum()

0

scores dataset has no missing values or duplicated rows

In [27]:
genome_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


In [28]:
missing_values = genome_tags.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [29]:
genome_tags.duplicated().sum()

0

In [30]:
link.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  27278 non-null  int64  
 1   imdbId   27278 non-null  int64  
 2   tmdbId   27026 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 639.5 KB


In [31]:
missing_values = link.isnull().sum()
missing_values[missing_values > 0]

tmdbId    252
dtype: int64

In [32]:
link.duplicated().sum()

0

In [33]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


In [34]:
movie.duplicated().sum()

0

In [35]:
missing_values = movie.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [36]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 610.4+ MB


In [37]:
missing_values = rating.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [38]:
rating.duplicated().sum()

0

In [75]:
rating = rating.rename(columns={'timestamp':'rating_timestamp'})
rating.head()

,userId,movieId,rating,rating_timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [58]:
highest_rated = rating.loc[rating['rating'].idxmax(), 'movieId']
lowest_rated = rating.loc[rating['rating'].idxmin(), 'movieId']
#reading titles from movie dataset
print(f'The highest rated movie is {movie.loc[movie['movieId'] == highest_rated, 'title' ].values[0]} ')
print(f'The lowest rated movie is {movie.loc[movie['movieId'] == lowest_rated, 'title'].values[0]} ')

The highest rated movie is Lord of the Rings: The Fellowship of the Ring, The (2001) 
The lowest rated movie is Nemesis 2: Nebula (1995) 


In [ ]:
# splitting genres column to do store it as a list
movie['genres'] = movie['genres'].str.split('|')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [39]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465564 entries, 0 to 465563
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   userId     465564 non-null  int64 
 1   movieId    465564 non-null  int64 
 2   tag        465548 non-null  object
 3   timestamp  465564 non-null  object
dtypes: int64(2), object(2)
memory usage: 14.2+ MB


In [ ]:
missing_values = tags.isnull().sum()
missing_values[missing_values > 0]
#we could drop this rows or just keep them to check which user did not give a tag for which movies

tag    16
dtype: int64

In [41]:
tags.duplicated().sum()

0

In [76]:
tags = tags.rename(columns={'timestamp':'tag_timestamp'})
tags.head()

,userId,movieId,tag,tag_timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


### Merging the datasets on common columns

In [ ]:
#merging tags and genom_tags on 'tag' column
merged_tags = pd.merge(tags, genome_tags, on ='tag', how = 'left')
merged_tags.head()

,userId,movieId,tag,tag_timestamp,tagId
0,18,4141,Mark Waters,2009-04-24 18:19:40,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288.0
2,65,353,dark hero,2013-05-10 01:41:19,288.0
3,65,521,noir thriller,2013-05-10 01:39:43,712.0
4,65,592,dark hero,2013-05-10 01:41:18,288.0


In [78]:
#tag ids are converted into floats because od Nan values
#let's convert them back to nullable ints
merged_tags['tagId'] = merged_tags['tagId'].astype('Int64')

In [79]:
merged_tags.head()

,userId,movieId,tag,tag_timestamp,tagId
0,18,4141,Mark Waters,2009-04-24 18:19:40,<NA>
1,65,208,dark hero,2013-05-10 01:41:18,288
2,65,353,dark hero,2013-05-10 01:41:19,288
3,65,521,noir thriller,2013-05-10 01:39:43,712
4,65,592,dark hero,2013-05-10 01:41:18,288


In [80]:
# joining the genome_scores to add the tag relevance
merged_tag = pd.merge(merged_tags, genome_scores, on = ['movieId', 'tagId'], how='left')

In [82]:
merged_tag.head()

,userId,movieId,tag,tag_timestamp,tagId,relevance
0,18,4141,Mark Waters,2009-04-24 18:19:40,<NA>,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288,0.94950
2,65,353,dark hero,2013-05-10 01:41:19,288,0.99800
3,65,521,noir thriller,2013-05-10 01:39:43,712,0.98900
4,65,592,dark hero,2013-05-10 01:41:18,288,0.99775


In [83]:
df = pd.merge(merged_tag, rating, on = ['userId', 'movieId'], how = 'left')
df.head()

,userId,movieId,tag,tag_timestamp,tagId,relevance,rating,rating_timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40,<NA>,NaN,NaN,NaN
1,65,208,dark hero,2013-05-10 01:41:18,288,0.94950,NaN,NaN
2,65,353,dark hero,2013-05-10 01:41:19,288,0.99800,NaN,NaN
3,65,521,noir thriller,2013-05-10 01:39:43,712,0.98900,NaN,NaN
4,65,592,dark hero,2013-05-10 01:41:18,288,0.99775,NaN,NaN


In [ ]:
df = pd.merge(movie, df, on ='movieId', how='left')

In [91]:
df['userId'] = df['userId'].astype('Int64')
df.head()

,movieId,title,genres,userId,tag,tag_timestamp,tagId,relevance,rating,rating_timestamp
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1644,Watched,2014-12-04 23:44:40,<NA>,NaN,3.5,2015-01-13 07:47:23
1,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1741,computer animation,2007-07-08 13:59:15,244,0.9985,4.0,2002-11-27 18:31:23
2,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1741,Disney animated feature,2007-07-08 22:21:47,<NA>,NaN,4.0,2002-11-27 18:31:23
3,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1741,Pixar animation,2007-07-08 22:46:10,<NA>,NaN,4.0,2002-11-27 18:31:23
4,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1741,TÃ©a Leoni does not star in this movie,2009-06-15 19:19:33,<NA>,NaN,4.0,2002-11-27 18:31:23


In [92]:
df.to_csv('complete_dataset.csv', index=False)

In [ ]:
'''
only at the final df converting timestamps into different columns for better readability
date_time = pd.to_datetime(df['....timestamp'])
import calendar
df['year'] = date_time.dt.year
df['month'] = date_time.dt.month
df['day'] = date_time.dt.day
df['time'] = date_time.dt.time
'''

## 1.2 Minhash Signatures

## 1.3 Locality-Sensitive Hashing (LSH)

### 1. Bucket Creation

### 2. Query

### 3. Movie Recommendation Logic

### 4. Final Recommendation

# 2. Grouping Movies Together!

## 2.1 Feature Engineering

## 2.2 Choose your features (variables)!

## 2.3 Clustering

## 2.4 Best Algorithm

# 3. Bonus Question

# 4. Algorithmic Question